# 14 – ETL de FAOSTAT: Cost and Affordability of a Healthy Diet (UE-27, 2017–2023)

En este notebook se transforma el fichero CSV descargado de FAOSTAT:

`data_raw/faostat/FAOSTAT_data_en_12-9-2025.csv`

El fichero pertenece al dominio:

- CAHD – Cost and Affordability of a Healthy Diet (CoAHD)

y contiene, para cada país y año, distintos indicadores relacionados con:

- Cost of a healthy diet (CoHD), LCU per person per day.
- Otros indicadores derivados del coste y asequibilidad.

### Objetivos del ETL

Generar un dataset limpio y en formato columnar (Parquet) que:

- Esté filtrado a:

  - Países de la Unión Europea.
  - Años 2017–2023 (intersección útil con el resto de fuentes).

- Mantenga la estructura típica de FAOSTAT:

  - País / área (`Area`, código M49).
  - Indicador (`Item`).
  - Año (`Year`).
  - Unidad (`Unit`).
  - Valor numérico (`Value`).

- Añada un código de país compatible con Eurostat / OFF:

  - `geo`: código ISO2 tipo Eurostat (AT, BE, ES, …, SE).

- Limpie el valor numérico:

  - Conversión de `Value` (VARCHAR) a `DOUBLE`.
  - Tratamiento de posibles vacíos / valores especiales como `NULL`.

El resultado se guardará en:

`data_processed/faostat/faostat_cahd_eu27_2017_2023.parquet`

In [1]:
from pathlib import Path
import duckdb
import pandas as pd

# Carpeta raíz del proyecto
ROOT_DIR = Path("..").resolve().parent

# Carpetas de datos crudos y procesados de FAOSTAT
DATA_RAW = ROOT_DIR / "data_raw" / "faostat"
DATA_PROCESSED = ROOT_DIR / "data_processed" / "faostat"

# Fichero CSV original descargado de FAOSTAT (CoAHD)
FAOSTAT_CSV = DATA_RAW / "FAOSTAT_data_en_12-9-2025.csv"

# Fichero de salida (Parquet limpio, UE-27, 2017–2023)
FAOSTAT_PARQUET = DATA_PROCESSED / "faostat_cahd_eu27_2017_2023.parquet"

ROOT_DIR, FAOSTAT_CSV.exists(), FAOSTAT_CSV, FAOSTAT_PARQUET

(WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm'),
 True,
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_raw/faostat/FAOSTAT_data_en_12-9-2025.csv'),
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_processed/faostat/faostat_cahd_eu27_2017_2023.parquet'))

In [2]:
# Conexión en memoria a DuckDB
con = duckdb.connect(database=":memory:")

# Vista rápida del CSV de FAOSTAT
preview = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
    LIMIT 5
""").fetchdf()

preview

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Release,Unit,Value,Flag,Flag Description
0,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),040,Austria,6205,Value,70041,"Cost of a healthy diet (CoHD), LCU per person ...",2017,2017,July 2025 (SOFI report),LCU/cap/d,1.72,E,Estimated value
1,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),040,Austria,6205,Value,70041,"Cost of a healthy diet (CoHD), LCU per person ...",2018,2018,July 2025 (SOFI report),LCU/cap/d,1.75,E,Estimated value
2,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),040,Austria,6205,Value,70041,"Cost of a healthy diet (CoHD), LCU per person ...",2019,2019,July 2025 (SOFI report),LCU/cap/d,1.77,E,Estimated value
3,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),040,Austria,6205,Value,70041,"Cost of a healthy diet (CoHD), LCU per person ...",2020,2020,July 2025 (SOFI report),LCU/cap/d,1.81,E,Estimated value
4,CAHD,Cost and Affordability of a Healthy Diet (CoAHD),040,Austria,6205,Value,70041,"Cost of a healthy diet (CoHD), LCU per person ...",2021,2021,July 2025 (SOFI report),LCU/cap/d,1.82,E,Estimated value


In [3]:
# Esquema de columnas del CSV de FAOSTAT
schema_df = con.execute(f"""
    DESCRIBE
    SELECT *
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
""").fetchdf()

schema_df

,column_name,column_type,null,key,default,extra
0,Domain Code,VARCHAR,YES,None,None,None
1,Domain,VARCHAR,YES,None,None,None
2,Area Code (M49),VARCHAR,YES,None,None,None
3,Area,VARCHAR,YES,None,None,None
4,Element Code,BIGINT,YES,None,None,None
5,Element,VARCHAR,YES,None,None,None
6,Item Code,BIGINT,YES,None,None,None
7,Item,VARCHAR,YES,None,None,None
8,Year Code,BIGINT,YES,None,None,None
9,Year,BIGINT,YES,None,None,None


In [4]:
# Resumen básico de años, áreas, items y elementos
basic_stats = con.execute(f"""
    SELECT
        MIN("Year") AS min_year,
        MAX("Year") AS max_year,
        COUNT(*)     AS n_filas,
        COUNT(DISTINCT "Area")    AS n_areas,
        COUNT(DISTINCT "Item")    AS n_items,
        COUNT(DISTINCT "Element") AS n_elements
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
""").fetchdf()

basic_stats

,min_year,max_year,n_filas,n_areas,n_items,n_elements
0,2017,2024,1512,27,16,1


In [5]:
# Items (indicadores) más frecuentes en el dominio CAHD
items_df = con.execute(f"""
    SELECT "Item", "Unit", COUNT(*) AS n
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
    GROUP BY "Item", "Unit"
    ORDER BY n DESC
    LIMIT 20
""").fetchdf()

items_df

,Item,Unit,n
0,"Prevalence of unaffordability (PUA), percent",%,378
1,Number of people unable to afford a healthy di...,million No,378
2,"Cost of a healthy diet (CoHD), PPP dollar per ...",Int$ (PPP) per person per day,216
3,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,216
4,"Cost of fruits, LCU per person per day",LCU/cap/d,27
5,"Cost of oils and fats, PPP dollar per person p...",Int$ (PPP) per person per day,27
6,"Cost of vegetables, PPP dollar per person per day",Int$ (PPP) per person per day,27
7,"Cost of legumes, nuts and seeds, LCU per perso...",LCU/cap/d,27
8,"Cost of starchy staples, PPP dollar per person...",Int$ (PPP) per person per day,27
9,"Cost of animal source foods, PPP dollar per pe...",Int$ (PPP) per person per day,27


In [6]:
# Áreas (países) más frecuentes
areas_df = con.execute(f"""
    SELECT "Area", COUNT(*) AS n
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
    GROUP BY "Area"
    ORDER BY n DESC
    LIMIT 30
""").fetchdf()

areas_df

,Area,n
0,Germany,56
1,Slovakia,56
2,Denmark,56
3,Hungary,56
4,Czechia,56
5,Bulgaria,56
6,Portugal,56
7,Romania,56
8,Sweden,56
9,Lithuania,56


In [14]:
# Cobertura específica del periodo 2017–2023
stats_2017_2023 = con.execute(f"""
    SELECT
        SUM( ("Year" BETWEEN 2017 AND 2023)::INT ) AS n_2017_2023,
        COUNT(*) AS n_total
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
""").fetchdf()

### Definición de países UE-27 y correspondencia con código ISO2 (Eurostat)

FAOSTAT utiliza nombres de país en inglés (Austria, Belgium, Spain, …), mientras que en Eurostat el código `geo` se expresa como:

- AT, BE, BG, HR, CY, CZ, DK, EE, FI, FR, DE, EL,
- HU, IE, IT, LV, LT, LU, MT, NL, PL, PT, RO, SK,
- SI, ES, SE.

En este ETL se construye una correspondencia explícita:

- `Area` (FAOSTAT) → `geo` (código ISO2 estilo Eurostat).

Esto permitirá después cruzar FAOSTAT con HICP, HBS y OFF.

In [15]:
# Comprobación rápida: lista de áreas distintas en el CSV (para referencia)
all_areas = con.execute(f"""
    SELECT DISTINCT "Area"
    FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
    ORDER BY "Area"
""").fetchdf()

all_areas.head(30)

,Area
0,Austria
1,Belgium
2,Bulgaria
3,Croatia
4,Cyprus
5,Czechia
6,Denmark
7,Estonia
8,Finland
9,France


In [16]:
# Aseguramos carpeta de salida para los datos procesados
FAOSTAT_PARQUET.parent.mkdir(parents=True, exist_ok=True)

# ETL principal:
#   - Filtrar dominio CAHD.
#   - Limpiar columna Value -> DOUBLE.
#   - Filtrar UE-27 mediante mapeo Area -> geo (código ISO2).
#   - Limitar a años 2017–2023.
con.execute(f"""
    COPY (
        WITH base AS (
            SELECT
                "Domain Code"        AS domain_code,
                "Domain"             AS domain,
                "Area Code (M49)"    AS area_m49,
                "Area"               AS area,
                "Element Code"       AS element_code,
                "Element"            AS element,
                "Item Code"          AS item_code,
                "Item"               AS item,
                "Year"               AS year,
                "Unit"               AS unit,
                "Value"              AS value_raw,
                "Flag"               AS flag,
                "Flag Description"   AS flag_desc
            FROM read_csv_auto('{FAOSTAT_CSV}', header=TRUE)
            WHERE "Domain Code" = 'CAHD'
        ),
        clean AS (
            SELECT
                domain_code,
                domain,
                area_m49,
                area,
                element_code,
                element,
                item_code,
                item,
                CAST(year AS INTEGER) AS year,
                unit,
                -- Limpieza de Value:
                --   - TRIM para quitar espacios.
                --   - Eliminamos el símbolo '<' (p.ej. '<0.1' -> '0.1').
                --   - '' y '..' -> NULL.
                --   - conversión a DOUBLE.
                NULLIF(
                    NULLIF(
                        REPLACE(TRIM(value_raw), '<', ''),
                        ''
                    ),
                    '..'
                )::DOUBLE AS value,
                flag,
                flag_desc
            FROM base
        ),
        eu27 AS (
            SELECT
                c.domain_code,
                c.domain,
                c.area_m49,
                c.area,
                m.iso2 AS geo,   -- código ISO2 compatible con Eurostat
                c.element_code,
                c.element,
                c.item_code,
                c.item,
                c.year,
                c.unit,
                c.value,
                c.flag,
                c.flag_desc
            FROM clean c
            JOIN (
                VALUES
                    ('Austria',                             'AT'),
                    ('Belgium',                             'BE'),
                    ('Bulgaria',                            'BG'),
                    ('Croatia',                             'HR'),
                    ('Cyprus',                              'CY'),
                    ('Czechia',                             'CZ'),
                    ('Denmark',                             'DK'),
                    ('Estonia',                             'EE'),
                    ('Finland',                             'FI'),
                    ('France',                              'FR'),
                    ('Germany',                             'DE'),
                    ('Greece',                              'EL'),
                    ('Hungary',                             'HU'),
                    ('Ireland',                             'IE'),
                    ('Italy',                               'IT'),
                    ('Latvia',                              'LV'),
                    ('Lithuania',                           'LT'),
                    ('Luxembourg',                          'LU'),
                    ('Malta',                               'MT'),
                    ('Netherlands (Kingdom of the)',        'NL'),
                    ('Poland',                              'PL'),
                    ('Portugal',                            'PT'),
                    ('Romania',                             'RO'),
                    ('Slovakia',                            'SK'),
                    ('Slovenia',                            'SI'),
                    ('Spain',                               'ES'),
                    ('Sweden',                              'SE')
            ) AS m(area, iso2)
              ON c.area = m.area
            WHERE c.year BETWEEN 2017 AND 2023
        )
        SELECT
            domain_code,
            domain,
            area_m49,
            area,
            geo,
            element_code,
            element,
            item_code,
            item,
            year,
            unit,
            value,
            flag,
            flag_desc
        FROM eu27
    )
    TO '{FAOSTAT_PARQUET}'
    (FORMAT PARQUET)
""")

FAOSTAT_PARQUET.exists(), FAOSTAT_PARQUET

(True,
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_processed/faostat/faostat_cahd_eu27_2017_2023.parquet'))

In [17]:
# Muestra de control del Parquet resultante
faostat_sample = con.execute(f"""
    SELECT
        geo,
        area,
        year,
        item,
        unit,
        value,
        flag
    FROM read_parquet('{FAOSTAT_PARQUET}')
    ORDER BY geo, item, year
    LIMIT 20
""").fetchdf()

faostat_sample

,geo,area,year,item,unit,value,flag
0,AT,Austria,2017,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,1.72,E
1,AT,Austria,2018,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,1.75,E
2,AT,Austria,2019,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,1.77,E
3,AT,Austria,2020,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,1.81,E
4,AT,Austria,2021,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,1.82,E
5,AT,Austria,2022,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,2.02,E
6,AT,Austria,2023,"Cost of a healthy diet (CoHD), LCU per person ...",LCU/cap/d,2.24,E
7,AT,Austria,2017,"Cost of a healthy diet (CoHD), PPP dollar per ...",Int$ (PPP) per person per day,2.06,E
8,AT,Austria,2018,"Cost of a healthy diet (CoHD), PPP dollar per ...",Int$ (PPP) per person per day,2.11,E
9,AT,Austria,2019,"Cost of a healthy diet (CoHD), PPP dollar per ...",Int$ (PPP) per person per day,2.19,E


In [18]:
# Resumen del dataset procesado: rango de años, nº filas, países, items
faostat_stats = con.execute(f"""
    SELECT
        MIN(year)               AS min_year,
        MAX(year)               AS max_year,
        COUNT(*)                AS n_filas,
        COUNT(DISTINCT geo)     AS n_paises,
        COUNT(DISTINCT item)    AS n_items,
        COUNT(DISTINCT unit)    AS n_units
    FROM read_parquet('{FAOSTAT_PARQUET}')
""").fetchdf()

faostat_stats

,min_year,max_year,n_filas,n_paises,n_items,n_units
0,2017,2023,1404,27,16,4


In [19]:
# Esquema del Parquet de salida
faostat_schema_out = con.execute(f"""
    DESCRIBE
    SELECT *
    FROM read_parquet('{FAOSTAT_PARQUET}')
""").fetchdf()

faostat_schema_out

,column_name,column_type,null,key,default,extra
0,domain_code,VARCHAR,YES,None,None,None
1,domain,VARCHAR,YES,None,None,None
2,area_m49,VARCHAR,YES,None,None,None
3,area,VARCHAR,YES,None,None,None
4,geo,VARCHAR,YES,None,None,None
5,element_code,BIGINT,YES,None,None,None
6,element,VARCHAR,YES,None,None,None
7,item_code,BIGINT,YES,None,None,None
8,item,VARCHAR,YES,None,None,None
9,year,INTEGER,YES,None,None,None
